In [45]:
# https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
# multivariate lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Input
import numpy as np

In [46]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [47]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
input_seq= hstack((in_seq1, in_seq2)) # row,column format
output_seq = out_seq.reshape((len(out_seq), 1))

In [48]:
input_seq

array([[10, 15],
       [20, 25],
       [30, 35],
       [40, 45],
       [50, 55],
       [60, 65],
       [70, 75],
       [80, 85],
       [90, 95]])

In [49]:
output_seq

array([[ 25],
       [ 45],
       [ 65],
       [ 85],
       [105],
       [125],
       [145],
       [165],
       [185]])

In [50]:
n_steps = 3
n_features = 2

# convert into input window / next output
#X, y = split_sequences(input_seq,output_seq, n_steps)

X = np.lib.stride_tricks.sliding_window_view(hstack((in_seq1, in_seq2)), window_shape= (n_steps,n_features))
X = np.squeeze(X, axis=1) # Para eliminar eje y que no aparezca (7,1,3,2) sino (7,3,2)
y = out_seq[n_steps-1:]  # Lo que predices (el siguiente valor después de cada ventana)

print(X.shape)
print(y.shape)

(7, 3, 2)
(7,)


In [51]:
# define model
model = Sequential()
model.add(Input(shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [52]:
# fit model
model.fit(X, y, epochs=200)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 14637.2490
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 14413.9463
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 14190.7178
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 13970.9199
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 13755.8857
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 13545.6104
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 13338.7969
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 13135.6865
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 12937.6279
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 12745.6113
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 12558.9297
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 12374.3926
Epoch 13/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 12190.9922
Epoch 14/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 12007.4990
Epoch 15/200
1/1 ━━━━━━━━━━━━━━━

In [53]:
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[205.74449]]
